In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import re
import string
import json
pd.options.mode.chained_assignment = None  # default='warn'
from merge_vegas import open_pickle, save_to_pickle
from import_yelp_mongo import get_yelp_reviews, get_yelp_reviews_afterdate

%matplotlib inline

/Users/tracy/anaconda/lib/python2.7/site-packages/fuzzywuzzy/fuzz.py:33: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
from IPython.display import display

# Read in datasets

In [323]:
WI = open_pickle('../data/mad/madison_yelp_merge.pkl')
NV = open_pickle('../data/vegas/vegas_yelp_merge.pkl')
AZ = open_pickle('../data/phx/phoenix_yelp_merge.pkl')
NC = open_pickle('../data/char/charlotte_yelp_merge.pkl')

In [324]:
print WI.shape
print NV.shape
print AZ.shape
print NC.shape

(654, 54)
(952, 40)
(3984, 43)
(1332, 56)


# Subset Review database

In [14]:
def state_yelp_reviews(df, d, tag):
    ids_ = df.business_id.values.tolist()
    R = get_yelp_reviews_afterdate(ids_, d)
    print R.info()
    
    save_to_pickle(R, '../data/yelp/yelp_reviews_%s.pkl' % tag)
    
    return R

In [15]:
R_WI = state_yelp_reviews(WI, '2011-06-30', 'madison')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18984 entries, 0 to 18983
Data columns (total 9 columns):
_id            18984 non-null object
business_id    18984 non-null object
date           18984 non-null object
review_id      18984 non-null object
stars          18984 non-null int64
text           18984 non-null object
type           18984 non-null object
user_id        18984 non-null object
votes          18984 non-null object
dtypes: int64(1), object(8)
memory usage: 1.4+ MB
None


## Madison, WI

In [335]:
import features_madison as lib

In [342]:
reload(lib)

<module 'features_madison' from 'features_madison.pyc'>

In [6]:
WI.info()

<class 'pandas.core.frame.DataFrame'>
Index: 654 entries, (43 north, 108, king st, madison, 53703) to (windsor, 53598)
Data columns (total 54 columns):
address                    654 non-null object
addressy_                  654 non-null object
attributes                 654 non-null object
avg                        356 non-null float64
avg_2                      356 non-null float64
avg_3                      356 non-null float64
avg_w                      356 non-null float64
avg_w_2                    356 non-null float64
avg_w_3                    356 non-null float64
business_id                654 non-null object
categories                 654 non-null object
city                       654 non-null object
complex                    654 non-null object
complexy_                  654 non-null object
full_address               654 non-null object
fuzz_partial_ratio         356 non-null float64
fuzz_partial_ratio_2       356 non-null float64
fuzz_partial_ratio_3       356 non-null f

In [17]:
print R_WI.date.min()
print R_WI.date.max()

2011-07-01
2015-01-08


In [385]:
print pd.to_datetime(I_WI.date).min()
print pd.to_datetime(I_WI.date).max()

2012-01-04 00:00:00
2015-10-30 00:00:00


In [343]:
%%time
df_WI = lib.get_features_WI(WI)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18984 entries, 0 to 18983
Data columns (total 9 columns):
_id            18984 non-null object
business_id    18984 non-null object
date           18984 non-null object
review_id      18984 non-null object
stars          18984 non-null int64
text           18984 non-null object
type           18984 non-null object
user_id        18984 non-null object
votes          18984 non-null object
dtypes: int64(1), object(8)
memory usage: 1.4+ MB
None
(94353, 15)
(16775, 15)
CPU times: user 53.8 s, sys: 7.5 s, total: 1min 1s
Wall time: 1min 19s


In [269]:
save_to_pickle(df_WI, '../data/mad/madison_yelp_features.pkl')

# Charlotte, NC

In [354]:
import features_charlotte as lib

In [398]:
reload(lib)

<module 'features_charlotte' from 'features_charlotte.py'>

In [346]:
NC.columns.values

array(['address', 'addressy_', u'attributes', 'avg', 'avg_2', 'avg_3',
       'avg_w', 'avg_w_2', 'avg_w_3', u'business_id', u'categories',
       'city', u'cityy_', 'complex', 'county', u'full_address',
       'fuzz_partial_ratio', 'fuzz_partial_ratio_2',
       'fuzz_partial_ratio_3', 'fuzz_ratio', 'fuzz_ratio_2',
       'fuzz_ratio_3', 'fuzz_token_set_ratio', 'fuzz_token_set_ratio_2',
       'fuzz_token_set_ratio_3', 'fuzz_token_sort_ratio',
       'fuzz_token_sort_ratio_2', 'fuzz_token_sort_ratio_3', u'hours',
       'id_', 'inspections', u'latitude', u'longitude', 'max', 'max_2',
       'max_3', 'name', 'name2_h', 'name2_y', 'name_', 'name_y_',
       u'namey_', u'neighborhoods', 'num', 'numy_', u'open',
       u'review_count', u'stars', 'state', u'statey_', 'street',
       'streety_', 'suite', 'suitey_', 'type', u'typey_'], dtype=object)

In [382]:
I_NC = lib.get_NC_inspections(NC, drop_flag=False)

In [383]:
I_NC.head()

,date,score,grade,inspector,id_,inspec_id
0,3/3/2015,97,A,2219,161416,161416_0
1,12/5/2014,96,A,2219,161416,161416_1
2,9/24/2015,97,A,2427,127669,127669_0
3,5/26/2015,97.50,A,2447,127669,127669_1
4,1/15/2015,97,A,2447,127669,127669_2


In [386]:
print pd.to_datetime(I_NC.date).min()
print pd.to_datetime(I_NC.date).max()

2013-06-24 00:00:00
2015-10-21 00:00:00


In [387]:
R_NC = lib.state_yelp_reviews(NC, '2012-11-30', 'charlotte')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29898 entries, 0 to 29897
Data columns (total 9 columns):
_id            29898 non-null object
business_id    29898 non-null object
date           29898 non-null object
review_id      29898 non-null object
stars          29898 non-null int64
text           29898 non-null object
type           29898 non-null object
user_id        29898 non-null object
votes          29898 non-null object
dtypes: int64(1), object(8)
memory usage: 2.3+ MB
None


In [391]:
y, x = lib.merge_inspec_dates(I_NC, NC, R_NC, ['score','grade'])

(175773, 15)
(130504, 15)


In [399]:
%%time
df_NC = lib.get_features_NC(NC, '2011-06-30', 'charlotte', ['score','grade'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41485 entries, 0 to 41484
Data columns (total 9 columns):
_id            41485 non-null object
business_id    41485 non-null object
date           41485 non-null object
review_id      41485 non-null object
stars          41485 non-null int64
text           41485 non-null object
type           41485 non-null object
user_id        41485 non-null object
votes          41485 non-null object
dtypes: int64(1), object(8)
memory usage: 3.2+ MB
None
(241574, 15)
(196930, 15)
CPU times: user 57.9 s, sys: 2.71 s, total: 1min
Wall time: 1min 23s


# Model, Create X, y:

In [285]:
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import f1_score

In [278]:
df_train, df_test = train_test_split(df_WI)

X_train = df_train[['rev_ct','neg_ct','stars_avg','rev_len_avg','stars_var']].values
y_train = df_train[['n_violations','n_critical']].values
X_test = df_test[['rev_ct','neg_ct','stars_avg','rev_len_avg','stars_var']].values
y_test = df_test[['n_violations','n_critical']].values

In [276]:
model_ols = LinearRegression()
model_ols.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [297]:
model_ols_sm = sm.OLS(y_train[:,1], sm.add_constant(X_train[:,[2,3,4]]) )
res = model_ols_sm.fit()
print res.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     5.739
Date:                Wed, 11 Nov 2015   Prob (F-statistic):           0.000667
Time:                        13:10:02   Log-Likelihood:                -3164.3
No. Observations:                1485   AIC:                             6337.
Df Residuals:                    1481   BIC:                             6358.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          1.8800      0.248      7.585      0.0

In [283]:
df_train.describe()

,n_violations,n_critical,rev_ct,neg_ct,stars_avg,rev_len_avg,stars_var
count,1485.000000,1485.000000,1485.000000,1485.000000,1485.000000,1485.000000,1485.000000
mean,4.282155,2.220202,8.716498,8.716498,3.502187,597.791884,1.111015
std,3.546835,2.050408,12.690321,12.690321,0.939607,328.411301,1.203548
min,0.000000,0.000000,1.000000,1.000000,1.000000,36.000000,0.000000
25%,2.000000,1.000000,2.000000,2.000000,3.000000,391.000000,0.000000
50%,4.000000,2.000000,4.000000,4.000000,3.666667,564.750000,0.894737
75%,6.000000,3.000000,10.000000,10.000000,4.100000,740.812500,1.655556
max,22.000000,13.000000,134.000000,134.000000,5.000000,4236.000000,8.000000


In [284]:
df_test.describe()

,n_violations,n_critical,rev_ct,neg_ct,stars_avg,rev_len_avg,stars_var
count,495.000000,495.000000,495.000000,495.000000,495.000000,495.000000,495.000000
mean,4.454545,2.430303,7.739394,7.739394,3.491642,631.490225,1.044846
std,4.076378,2.432633,9.661412,9.661412,0.949514,384.902831,1.123696
min,0.000000,0.000000,1.000000,1.000000,1.000000,38.000000,0.000000
25%,2.000000,1.000000,2.000000,2.000000,3.000000,390.250000,0.000000
50%,3.000000,2.000000,4.000000,4.000000,3.666667,575.250000,0.787879
75%,6.000000,4.000000,10.000000,10.000000,4.088008,784.083333,1.600000
max,37.000000,17.000000,65.000000,65.000000,5.000000,4242.000000,8.000000


In [344]:
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train[:,1])
y_pred = model_rf.predict(X_test)
#f1_score(y_test[:,1]+0.001, y_pred+0.001)

In [298]:
I_WI.describe()

,n_violations,n_critical
count,2831.000000,2831.000000
mean,4.467326,2.374779
std,3.862419,2.280301
min,0.000000,0.000000
25%,1.000000,1.000000
50%,4.000000,2.000000
75%,6.000000,4.000000
max,37.000000,17.000000


In [313]:
temp = I_WI.copy()
temp['n_core'] = - temp.n_critical + temp.n_violations
temp['grade'] = 0
temp.loc[temp.n_critical>= 1,'grade'] +=1
temp.loc[temp.n_critical>= 2,'grade'] +=1
temp.loc[temp.n_critical>= 3,'grade'] +=1
temp.loc[temp.n_core>=4,'grade'] +=1
temp['target'] = temp.grade >= 3

In [322]:
temp.groupby('id_').max().describe()

,n_violations,n_critical,n_core,grade,target
count,636.000000,636.000000,636.000000,636.000000,636
mean,7.127358,3.753145,4.061321,2.886792,0.718553
std,4.149696,2.361850,2.710902,1.064716,0.450059
min,0.000000,0.000000,0.000000,0.000000,False
25%,4.000000,2.000000,2.000000,2.000000,0
50%,6.000000,3.000000,4.000000,3.000000,1
75%,9.000000,5.000000,6.000000,4.000000,1
max,37.000000,17.000000,20.000000,4.000000,True
